# Vocabulary of Unique Words

In [1]:
import pandas as pd
import os
import requests
import re
import gspread
from oauth2client.service_account import ServiceAccountCredentials


ModuleNotFoundError: No module named 'pandas'

In [ ]:
!pip install numpy


     |▎                               | 194kB 10kB/s eta 0:30:364

## Getting unique vocab from NLU

In [ ]:
def check_underscore_in_word(text):
    for c in text:
        if c=="_":
            return
    return text

In [6]:
def process_text(text):
  text = text.encode('ascii', errors='ignore').decode()
  text = text.lower()
  text = re.sub(r'http\S+', ' ', text)
  text = re.sub(r'#+', ' ', text )
  text = re.sub(r'@[A-Za-z0-9]+', ' ', text)
  text = re.sub(r"([A-Za-z]+)'s", r"\1 is", text)
  #text = re.sub(r"\'s", " ", text)
  text = re.sub(r"\'ve", " have ", text)
  text = re.sub(r"won't", "will not ", text)
  text = re.sub(r"isn't", "is not ", text)
  text = re.sub(r"can't", "can not ", text)
  text = re.sub(r"n't", " not ", text)
  text = re.sub(r"i'm", "i am ", text)
  text = re.sub(r"\'re", " are ", text)
  text = re.sub(r"\'d", " would ", text)
  text = re.sub(r"\'ll", " will ", text)
  text = re.sub('\W', ' ', text)
  text = re.sub(r'\d+', ' ', text)
  text = re.sub('\s+', ' ', text)
  text = text.strip()
  return text

In [11]:
!ls data/nlu

bi            faq           form.md       lookup_tables smalltalk


In [19]:
path_to_nlu_files = [
        "form.md",
        "bi/bi_faq.md",
        "bi/bi_quizes.md",
        "bi/bi.product.career_coaching.md",
        "bi/bi.product.learning_path.md",
        "bi/product.md",
        "faq/faq_data_science.md",
        "faq/faq_full_stack.md",
        "smalltalk/smalltalk.1.greetings.md",
        "smalltalk/smalltalk.2.emotions.md",
        "smalltalk/smalltalk.3.about_user.md",
        "smalltalk/smalltalk.4.courtesy.md",
        "smalltalk/smalltalk.5.about_chatbot.md"       
]

In [20]:
complete_content = ""
for file in path_to_nlu_files:
    f = open("./data/nlu/"+file,'r')
    file_content = f.read()
    complete_content = complete_content + file_content
complete_content

lk to you\n- do not want to talk to u\n- i did not want to talk\n- does not want to talk to you\n- do not want to talk\n- does not want to talk to u\n- do not talk\n- i does not want to talk\n- can we stop the discussion right now\n- end with the discussion\n- i do not like to talk now\n- i am not interested to talk to you\n- i am not interested to talk to u\n- hate to talk to u\n- not interested to talk\n- not interested to talk to u\n\n## intent:smalltalk.3.about_user.excited\n- i am excited\n- i am really excited\n- how excited i am\n- i am thrilled\n- i am excited about working with you\n- i am excited about working with u\n- i am excited to start our friendship\n- i am thrilled to talk to you\n- i am thrilled to talk to u\n- i am damn excited\n- i am hyped\n- i am very excited\n- i felt excited\n- it was exciting\n- thrilled\n- excited\n- ecstatic\n- that was exciting\n- how exciting\n- i am excited about tomorrow\n- it was very exciting i did like to use this feature again\n- i a

In [21]:
processed_complete_content = process_text(complete_content)
processed_complete_content

vo wow it was amazing wonderful it is amazing amazed woah it is amazing amazing wonderful it is wonderful it is so wonderful it is so amazing intent smalltalk about_user back i will get back to you in a moment be back in minutes i will be back soon i promise to come back i will be back in a few minutes i will right back i will brb brb back i will be back in a moment i will be back there in a moment i will be back there in a secs i will be back there in a minute will be back in secs will be back in a min will be back in a while i will be back i will be right back i will come back i shall be back i assure that i will be back soon i would be back man i will get back to u i will get back to you give me five sec i will return i will return shortly i will return in a jiffy when i return we will talk i will return in ten minutes i will be there in a moment i will be right back in a sec intent smalltalk about_user bored soo bored that was boring i am bored bored boring i am getting bored very 

In [22]:
vocab_nlu = set(processed_complete_content.split())
vocab_nlu

{'fields',
 'sure',
 'caring',
 'complaints',
 'klausimausi',
 'examine',
 'schedule',
 'finished',
 'juniors',
 'great',
 'explain',
 'deliver',
 'giving',
 'disagree',
 'your',
 'anser',
 'continue',
 'bi_faq',
 'unsure',
 'fresher',
 'hehe',
 'awfully',
 'soon',
 'honor',
 'phone_no',
 'type',
 'completing',
 'asked',
 'mood_unhappy',
 'centres',
 'becoming',
 'man',
 'requirements',
 'angels',
 'nor',
 'machine',
 'prannam',
 'undergraduate',
 'hm',
 'fees',
 'hope',
 'aap',
 'resume',
 'aapan',
 'request',
 'pune',
 'arbazkhan',
 'pipeline',
 'products',
 'information',
 'ya',
 'analysyis',
 'offered',
 'side',
 'set',
 'whatsup',
 'wasteland',
 'registration',
 'converse',
 'wht',
 'angel',
 'who_made_you',
 'lie',
 'applying',
 'scratch',
 'excite',
 'identify',
 'finance',
 'lmao',
 'job',
 'tc',
 'recognize',
 'question',
 'enter',
 'boyfriend',
 'discussion',
 'markjobs',
 'aspect',
 'secs',
 'real',
 'choosing',
 'freely',
 'wake',
 'se',
 'eligibility',
 'thrilled',
 'dream

In [23]:
len(vocab_nlu)

1634

### Removing the words with underscore

In [24]:
unique_vocab_nlu = []
for word in vocab_nlu:
    if '_' not in word:
#         print(word)
        unique_vocab_nlu.append(word)

In [25]:
len(unique_vocab_nlu)

1596

## Getting unique vocab from Domain FAQ

In [27]:
#############################################
# FETCHING INFO FROM GOOGLE SHEET 
#############################################

scope = ['https://spreadsheets.google.com/feeds',
 'https://www.googleapis.com/auth/drive']
## credentials json for authentications
credentials = ServiceAccountCredentials.from_json_keyfile_name('../googlesheetscredentials.json', scope)

gc = gspread.authorize(credentials)
wks = gc.open_by_key('1_t6qbAsc68hVxaPu6Xbmrsf91sANX3TiYnzlqhPo2HA').worksheet("FAQ")
data = wks.get_all_records()
complete_ds_faq = []
for i in range(len(data)):
    complete_ds_faq.append(data[i]["Answer"])

In [28]:
complete_ds_faq[:5]

['Big Data refers to the humongous amount of data which is tracked by businesses on a day to day basis. This data is majorly unstructured; and is generated by routing business processes running on a daily basis. The nature of Big Data in organizations is defined by <b>3Vs – Volume, Velocity & Variety</b>. ‘Volume’ refers to the large quantum of data in terabytes which may be transactional data, customer data etc. ‘Velocity’ refers to the rate at which the data is generated which could be either scheduled batch process, or real-time data collection. Finally, ‘variety’ denotes the variation in the data formats and structures.<br> <br> Data science, on the other hand, is a more specialized field which involves using mathematical & statistical modelling techniques on underlying data to devise patterns in data; and find actionable solutions to real-world problems.',
 'There’s not much of a difference between data scientist & business analysts in terms of the acquired skillsets & acumen; how

In [29]:
processed_ds_faq = ""
for sent in complete_ds_faq[:100]:
    processed_ds_faq = processed_ds_faq +" "+process_text(sent)
    print(processed_ds_faq)

e courses on r programming in addition board infinity would also guide the individual in making an attractive portfolio which could help him her get interesting freelance projects microsoft excel though being the most fundamental and basic tool for data analysis is used extensively primarily due to its flexibility ease of usage students can learn excel from variety of online sources such as the microsoft online community excel central excel hero mr excel and so on more importantly users can simply browse the help centre in the excel window to learn more about each of the formulae and functionalities used in excel a data scientist should have an optimal mix of data crunching mathematical acumen statistical know how and the requisite technical visualization skills to implement present the findings to end users assuming that you do not have any prior knowledge on programming or statistics it is important that you kickstart your journey with relatively simple programming languages such as 

In [30]:
unique_vocab_faq = set(processed_ds_faq.split())
unique_vocab_faq

{'fields',
 'touch',
 'cognizant',
 'matrix',
 'definition',
 'happenings',
 'considered',
 'directly',
 'databricks',
 'great',
 'speciality',
 'minimal',
 'deliver',
 'giving',
 'leverage',
 'underlying',
 'thin',
 'ideally',
 'your',
 'aspirants',
 'continue',
 'approximately',
 'profession',
 'slack',
 'mathematical',
 'optimal',
 'blended',
 'third',
 'experts',
 'across',
 'aligned',
 'blocks',
 'understands',
 'candidates',
 'motivation',
 'iims',
 'nature',
 'unclean',
 'becoming',
 'algebra',
 'hundred',
 'statistical',
 'structuring',
 'machine',
 'reading',
 'basis',
 'operate',
 'undergraduate',
 'resume',
 'agricultural',
 'financial',
 'route',
 'products',
 'representation',
 'tendency',
 'logical',
 'information',
 'place',
 'alternatives',
 'faults',
 'measures',
 'offered',
 'centric',
 'side',
 'set',
 'harmonized',
 'architecture',
 'returns',
 'responses',
 'limited',
 'actionable',
 'mentorship',
 'deployed',
 'applying',
 'identify',
 'finance',
 'job',
 'indicat

In [31]:
len(unique_vocab_faq)

1697

## Combining both the sets (Local NLU + Remote Sheets)

In [32]:
unique_vocab = list(unique_vocab_faq) + list(unique_vocab_nlu)
unique_vocab[:10]

['fields',
 'touch',
 'cognizant',
 'matrix',
 'definition',
 'happenings',
 'considered',
 'directly',
 'databricks',
 'great']

In [33]:
len(unique_vocab)

3293

In [34]:
unique_vocab

['fields',
 'touch',
 'cognizant',
 'matrix',
 'definition',
 'happenings',
 'considered',
 'directly',
 'databricks',
 'great',
 'speciality',
 'minimal',
 'deliver',
 'giving',
 'leverage',
 'underlying',
 'thin',
 'ideally',
 'your',
 'aspirants',
 'continue',
 'approximately',
 'profession',
 'slack',
 'mathematical',
 'optimal',
 'blended',
 'third',
 'experts',
 'across',
 'aligned',
 'blocks',
 'understands',
 'candidates',
 'motivation',
 'iims',
 'nature',
 'unclean',
 'becoming',
 'algebra',
 'hundred',
 'statistical',
 'structuring',
 'machine',
 'reading',
 'basis',
 'operate',
 'undergraduate',
 'resume',
 'agricultural',
 'financial',
 'route',
 'products',
 'representation',
 'tendency',
 'logical',
 'information',
 'place',
 'alternatives',
 'faults',
 'measures',
 'offered',
 'centric',
 'side',
 'set',
 'harmonized',
 'architecture',
 'returns',
 'responses',
 'limited',
 'actionable',
 'mentorship',
 'deployed',
 'applying',
 'identify',
 'finance',
 'job',
 'indicat

## Saving in file for Spell Checker Pipeline

In [36]:
with open('./chatbot_vocab.txt', 'w') as f:
    for item in unique_vocab:
        f.write("%s\n" % item)